## Extracting Elevation Data for Zip Codes using Google Earth Engine

This notebook demonstrates zonal statistics operations using [Google Earth Engine](https://earthengine.google.com/). To replicate, you will need to [sign up for an Earth Engine account](https://signup.earthengine.google.com/).  

Zonal statistics calculate descriptive statistics across a zone of a [raster](https://desktop.arcgis.com/en/arcmap/10.3/manage-data/raster-and-images/what-is-raster-data.htm) data set, such as a satellite image or a digital elevation model (DEM). The zones, in this case, are boundaries of Zip Code Tabulation Areas. See Esri's description of [How Zonal Statistics Works](https://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/h-how-zonal-statistics-works.htm).

The advantage of using Earth Engine for this is the ability to run the operation on the cloud instead of having to download and process a large amount of digital elevation models. Instead, Earth Engine has the [National Elevation Dataset (NED)](https://developers.google.com/earth-engine/datasets/catalog/USGS_NED) as well as [Zip Code Tabulation Areas (ZCTA)](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2010_ZCTA5) vector data which you can manipulate on the cloud. The NED resolution is 1/3 arc-second, or about 10 meters at mid-latitudes. 

This notebook uses the Earth Engine Python API. This requires some [setup](https://developers.google.com/earth-engine/guides/python_install-conda) and [credentials](https://developers.google.com/earth-engine/guides/python_install-conda#get_credentials), but it's free.

Note that you would find higher standard deviation for elevations within zip codes that are smaller in area but have a large elevation range. People that live in those zip codes probably live closer to the minimum, rather than the mean or maximum. 

#### Start by importing the following libraries:

In [2]:
import ee # earth engine
import pandas as pd
import os
import csv

##### You must initialize Earth Engine first:

In [3]:
ee.Initialize()

Here I navigate to a folder that contains a csv with about 600 zip codes:

In [5]:
os.chdir('C://users//phwh9568//Zips')

#### Make some variables out of the elevation and zip code data:

In [6]:
dataset = ee.Image('USGS/NED') # make a variable 'dataset' out of the USGS/NED dataset. 
elev = dataset.select('elevation') # select elevation from the NED dataset to a new variable 'elev'.
zips = ee.FeatureCollection('TIGER/2010/ZCTA5') # make 'zips' variable out of the ZCTA vecor data.
study_zips = pd.read_csv('med_zips_states.csv', dtype={'Zip':str}) # import the study-specific csv of zip codes into a pandas dataframe 

#### Make a list out of the study zip codes:
We will iterate over these.

In [7]:
zipList = study_zips.Zip.to_list()

#### Here we will open an output csv which we will write our output data into and iterate over the zip codes to calculate zonal statistics of elevation within each zip code.  
For each input zip code, the loop will:
1. Select the specific zip code's geometry from the ZCTA vector data
2. Use the Earth Engine [reduceRegion](https://developers.google.com/earth-engine/guides/reducers_reduce_region) function to perform a zonal statistics operation. This will use the boundary of a zip code and calculate descriptive statistics of the elevation data that falls within that zip code's boundaries.
3. Read the results
4. Write the results into the output csv

In [45]:
with open('output.csv', 'w', newline='', encoding='utf-8') as f:
    writer=csv.writer(f)
    writer.writerow(['Zip', 'MeanElev', 'MedianElev', 'ModeElev', 'MinElev', 'MaxElev', 'StdDevElev'])
    
    for zipCode in zipList:
        
        # use study zip to select corresponding vector geometry
        z = zips.filter("ZCTA5CE10=='{}'".format(zipCode))
        
        # compute mean elevation for the zip code
        elMean = elev.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry = z.geometry(),
            scale = 100)
        zipElMean= elMean.getInfo()['elevation']
        
        # compute median elevation for the zip code
        elMed = elev.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry = z.geometry(),
            scale = 100)
        zipElMed= elMed.getInfo()['elevation']

        # compute mode elevation for the zip code
        elMode = elev.reduceRegion(
            reducer=ee.Reducer.mode(),
            geometry = z.geometry(),
            scale = 100)
        zipElMode= elMode.getInfo()['elevation']
        
        # compute min & max elevation for the zip code
        elMinMax = elev.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry = z.geometry(),
            scale = 100)
        zipElMin= elMinMax.getInfo()['elevation_min']
        zipElMax= elMinMax.getInfo()['elevation_max']

        # compute standard deviation elevation for the zip code
        elStdDev = elev.reduceRegion(
            reducer=ee.Reducer.stdDev(),
            geometry = z.geometry(),
            scale = 100)
        zipElStdDev= elStdDev.getInfo()['elevation']
        
        # write the results to a new row in the output csv
        writer.writerow([zipCode, zipElMean, zipElMed, zipElMode, zipElMin, zipElMax, zipElStdDev])       

## Done!
#### Review the output.csv for the results.